# **KeyBERT 실습**

이번 노트북에서는 KeyBERT 모델을 한국어에 맞춰 사용할 수 있도록 Customizing한 후 인퍼런스 할 예정입니다. 인퍼런스 능력을 향상시키기 위해, 직접 BERT모델을 고르고, Tokenizer을 수정하며 주제를 merge하는 작업을 수행합니다. 이후, 결과 다양성을 높힐 수 있는 다양한 테크닉을 시도하고, 직접 결과를 확인합니다.

## **1 - 준비작업**

### **1.1 - 라이브러리 설치**
필요한 라이브러리를 설치합니다. 별도로 수정하지 말아주세요.

In [4]:
!pip install \
  torch \
  torchvision \
  torchaudio \
  --extra-index-url https://download.pytorch.org/whl/cu116

!pip install \
  keybert

!pip install \
  konlpy

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


^C


### **1.2 - 라이브러리 불러오기**
필요한 라이브러리를 불러옵니다. 별도로 수정하지 말아주세요.

In [73]:
import os
import csv
import numpy as np
import random
import torch
import pandas as pd
from tqdm import tqdm

from keybert import KeyBERT

import konlpy
from konlpy.tag import Komoran

from typing import List, Tuple, Dict, Literal, Optional, Callable

In [45]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

### **1.3 - Seed 고정**
결과를 일정하게 유지하기 위해 모든 SEED를 고정합니다. 별도로 수정하지 말아주세요.

In [9]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

### **1.4 - Colab**
코랩에 접근하기 위해 실행합니다. 그 전에, Colab 폴더에 코드와 데이터셋을 올바르게 넣었는지 확인해주세요.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## **2 - 데이터셋**

### **2.1 - 데이터셋 로드**
데이터셋을 드라이브에서 불러옵니다.

In [ ]:
PATH = './datasets/원자력+원전+탈원전_news_df.xlsx'
df1 = pd.read_excel(PATH)

PATH = './datasets/사설_반대.xlsx'
df2 = pd.read_excel(PATH)

PATH = './datasets/고리+신한울+새울+월성+신월성_news_df.xlsx'
df3 = pd.read_excel(PATH)

# df = pd.concat([df1, df2, df3], axis=0)

df = df2

미리보기

In [12]:
df.head()

,news_title,news_date,news_company,news_body
0,"[사설] `탈원전`, 방향은 맞지만 부작용 최소화해야",2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,[사설] 탈원전·석탄 조급증…안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,[fn사설] 새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,[사설] 탈원전…정규직화…'밀어붙이기 정책'에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,<사설>성급한 脫원전은 電力안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


### **2.2 - 텍스트 전처리**
해당 데이터는 크롤링된 데이터로, 올바른 결과를 위해서는 전처리가 필요합니다.
뉴스 기사를 위한 전처리 코드는 https://colab.research.google.com/drive/1O42v6YzHbTo_l36VG1xGp0aEuA0J8M3V?usp=sharing#scrollTo=P7JBkNna1jze 에서 가져왔습니다.

In [13]:
df['news_title'] = df['news_title'].map(lambda x: str(x))

In [14]:
import re, unicodedata
from string import whitespace
pattern_whitespace = re.compile(f'[{whitespace}]+')

df['news_title'] = df['news_title'].str.replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()


In [15]:
def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)
    result = pattern_others.sub('.', result)
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()

    return result

df['news_title'] = df['news_title'].map(clean_byline)

In [16]:
from string import whitespace, punctuation

def text_filter(text):
    punct_except_percent = ''.join([chr for chr in punctuation if chr != '%'])
    whitespace_convert_pattern = re.compile(f'[{whitespace}{punct_except_percent}]+')
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣]+')
    result = whitespace_convert_pattern.sub(' ', text)
    result = exclude_pattern.sub(' ', result).strip()
    result = whitespace_convert_pattern.sub(' ', result)
    return result

df['news_title'] = df['news_title'].map(text_filter)

전처리 후 결과 미리보기

In [17]:
df.head()

,news_title,news_date,news_company,news_body
0,탈원전 방향은 맞지만 부작용 최소화해야,2017-05-29,디지털타임스,29일 국정기획자문위원회 이개호 경제2분과 위원장이 원자력안전위원회(원안위)에 원전...
1,탈원전 석탄 조급증 안전하면서 값싼 전기는 없다,2017-05-30,한국경제,"문재인 정부의 에너지정책이 탈(脫)원전, 탈화력발전으로 급선회하고 있다. 그제 국정..."
2,새 정부 탈원전 정책 우려 새겨 들어야,2017-06-01,파이낸셜뉴스,전문가 200명 비판 성명 방향 맞지만 너무 서둘러 에너지 분야 전문가들이 ...
3,탈원전 정규직화 밀어붙이기 정책 에서 놓치는 것들,2017-06-01,한국경제,"교수 230명 성명 ""에너지정책 전문가 논의 절실""일자리 해법, 누구나 비판하고 토..."
4,성급한 원전은 안보 해친다는 전문가들의 우려,2017-06-02,문화일보,문재인 정부의 탈(脫)원전 정책에 대해 에너지 전문가들이 정면으로 비판하고 나섰다....


## **3 - 모델**

기본으로 제공하는 SBERT는 영어 문장을 대상으로만 문장을 분류, 군집화 할 수 있습니다. <br>
한국어를 활용해 SBERT를 수행하고 싶다면, 한국어를 지원하는 BERT모델을 사용해야합니다. <br>
한국어를 지원하는 SBERT모델은 [여기]()에서 찾을수 있습니다.<br>
(이때, 주의할 점은 문장간의 유사도를 판별할 수 있는 모델을 사용해야 하기 때문에, 꼭 일반적인 BERT가 아닌 SBERT를 사용해야 합니다!)

**HuggingFace에서 한국어 SBERT모델 찾는법** <br>
- Hugging Face > Models > Natural Language Processing(좌측 메뉴)에 들어간다
- Filter by name에 "ko"라고 검색
- 검색된 다양한 모델에 들어간 후 원하는 모델을 정합니다.

**모델을 고르는 방법** <br>
- 모델의 성능 (주관적이지만 기능을 수행하기에 무리가 없는 수준이어야 합니다.)
- 어떤 데이터셋으로 학습되었는가? (내가 사용할 Domain과 비슷한 Corpus로 학습된 모델이 좋습니다.)

실습은 Hugging Face의 "jhgan/ko-sroberta-multitask"을 사용해서 진행하겠습니다.

### **3.1 - 모델 불러오기**

그냥 모델을 간단히 불러와서 설정해도 되지만, 원하는 결과를 도출하기 위해서는 다양한 하이퍼파라미터를 직접 설정할 필요가 있습니다.
아래의 주석에 맞춰 차근차근 따라가보세요.

In [18]:
kw_model = KeyBERT(model='jhgan/ko-sroberta-multitask')

### **3.3 - 하이퍼파라미터 설정**

**설정해야하는 값**
- `keyphrase_ngram_range` : 토픽으로 사용할 키워드들의 ngram 길이를 설정 / (1, 2)면 1~2 글자로 이루어진 구를 대상으로 토픽 모델링을 진행
- `top_n` : 최대 몇개의 토픽을 추출할 것인지 설정
- `stop_words` : 불용어를 List로 전달

In [58]:
top_n = 5
keyphrase_ngram_range = (1, 1)

In [59]:
STOPWORDS_PATH = './datasets/user_stopwords.csv'

with open(STOPWORDS_PATH, 'r', encoding='cp949') as f:
    tmp = csv.reader(f)
    stop_words = set([l[0] for l in tmp])

## **4 - 키워드 추출하기**

이제 키워드를 추출해보겠습니다. 무거운 BERT모델을 사용하므로 꼭 GPU환경에서 실행하셔야하며, 30분 이상의 시간이 소요됩니다.

KeyBERT는 BERT의 기본 지식을 활용하므로, 각 문장마다 별도의 딜레이없이 바로바로 키워드를 추출할 수 있다는 장점이 있습니다.

5개의 문장만 추출해보겠습니다.

In [60]:
for text in df['news_title'][:5]:
    print(f"문장 : {text}")
    
    keywords = kw_model.extract_keywords(
        docs = text, 
        keyphrase_ngram_range = keyphrase_ngram_range, 
        stop_words = stop_words,
        top_n = top_n
        )
    print(f"토픽 : {keywords} \n")

문장 : 탈원전 방향은 맞지만 부작용 최소화해야
토픽 : [('탈원전', 0.7432), ('최소화해야', 0.5011), ('방향은', 0.2818), ('부작용', 0.2447), ('맞지만', 0.19)] 

문장 : 탈원전 석탄 조급증 안전하면서 값싼 전기는 없다
토픽 : [('탈원전', 0.602), ('전기는', 0.5102), ('값싼', 0.2545), ('안전하면서', 0.2506), ('조급증', 0.2364)] 

문장 : 새 정부 탈원전 정책 우려 새겨 들어야
토픽 : [('탈원전', 0.6832), ('우려', 0.2459), ('정부', 0.1332), ('들어야', 0.0941), ('새겨', 0.0499)] 

문장 : 탈원전 정규직화 밀어붙이기 정책 에서 놓치는 것들
토픽 : [('탈원전', 0.6304), ('정규직화', 0.4033), ('놓치는', 0.2757), ('밀어붙이기', 0.22), ('정책', 0.0637)] 

문장 : 성급한 원전은 안보 해친다는 전문가들의 우려
토픽 : [('원전은', 0.5054), ('우려', 0.3103), ('안보', 0.2958), ('해친다는', 0.2934), ('성급한', 0.2039)] 



## **5 - 전처리**

비슷한 키워드만 반복하고, 키워드가 부자연스러운 등 생각보다 결과가 불만족스럽습니다. 다양한 전처리를 적용한 후 다시 결과를 관찰해 보도록 하겠습니다.

### **5.1 - 품사 필터링**
KeyBERT는 기본적으로 띄어쓰기 단위로 N-gram을 계산한 후, 토픽 모델링을 진행합니다.<br>
하지만, 한국어는 여러가지 조사나 붙여쓰는 단어들이 섞여 있어 띄어쓰기 단위로 모델링 하는 것이 최적의 결과가 아닐 수 있습니다.<br>
따라서, 불필요한 품사를 제거하고, 명사, 형용사, 동사, 관형사, 일반부사만 사용하도록 필터링합니다.

(이때, 꼭 필요한 단어는 사용자 사전에 등록하도록합니다.)

In [95]:
USER_DICT_PATH = './datasets/user_dic.txt'

custom_dic = [
    '탈원전\tNNG',
]

with open(USER_DICT_PATH, 'w', encoding='utf-8') as f:
    for line in custom_dic:
        f.write(line + '\n')

In [98]:
tagger = Komoran(userdic='./datasets/user_dic.txt')
tag_list=['NNG', 'NNP', 'NNB', 'VV', 'VA', 'MM', 'MAG']

tokenized_text = []
for text in df['news_title']:
    word_tokens = ' '.join(map(lambda x: x[0], filter(lambda x: x[1] in tag_list, tagger.pos(text))))
    tokenized_text.append(word_tokens) 

In [103]:
tokenized_text[:4]

['탈원전 방향 맞 부작용 최소',
 '탈원전 석탄 조급증 안전 값싸 전기 없',
 '새 정부 탈원전 정책 우려 새기 들',
 '탈원전 정규 직 밀어붙이 정책 놓치 것']

결과를 보겠습니다.

In [100]:
for text in tokenized_text[:4]:
    print(f"문장 : {text}")
    
    keywords = kw_model.extract_keywords(
        docs = text, 
        keyphrase_ngram_range = keyphrase_ngram_range, 
        stop_words = stop_words,
        top_n = top_n
        )
    print(f"토픽 : {keywords} \n")

문장 : 탈원전 방향 맞 부작용 최소
토픽 : [('탈원전', 0.8296), ('방향', 0.3018), ('부작용', 0.2864), ('최소', 0.2789)] 

문장 : 탈원전 석탄 조급증 안전 값싸 전기 없
토픽 : [('탈원전', 0.6194), ('전기', 0.3925), ('값싸', 0.3398), ('안전', 0.2511), ('조급증', 0.2092)] 

문장 : 새 정부 탈원전 정책 우려 새기 들
토픽 : [('탈원전', 0.7299), ('우려', 0.239), ('정부', 0.1664), ('새기', -0.0242), ('정책', -0.0268)] 

문장 : 탈원전 정규 직 밀어붙이 정책 놓치 것
토픽 : [('탈원전', 0.7077), ('정규', 0.2489), ('놓치', 0.1947), ('밀어붙이', 0.1818), ('정책', 0.0447)] 



### **5.2 - 결과 다양성 조절하기**

위에서 모델링한 결과를 보면 비슷한 뜻을 가진 토픽이 상위에 반복적으로 노출되는 것을 볼 수 있습니다.<br>
이런 것들을 Max Sum Distance와 MMR(Max Margianl Relevance)를 통해 개선해보도록 하겠습니다.

**Max Sum Distance**

Max Sum Distance는 nr_candidates개 만큼의 키워드를 추출한 후, 그 중 top_n 조합을 생성하고, 조합 내 아이템의 유사도의 합이 가장 적은 것을 토픽으로 설정하는 방식입니다.

**설정해야하는 값**
- `use_maxsum` : True로 설정시 해당 옵션 사용
- `nr_candidates` : top_n 조합을 계산할 키워드 풀의 크기 / 높을수록 키워드 다양해짐 / 설정하지 않으면 2 * top_n

In [101]:
for text in tokenized_text[:4]:
    print(f"문장 : {text}")
    
    keywords = kw_model.extract_keywords(
        docs = text, 
        keyphrase_ngram_range = keyphrase_ngram_range, 
        stop_words = stop_words,
        top_n = top_n
        )
    print(f"원래 토픽 : {keywords}")

    keywords = kw_model.extract_keywords(
        docs = text, 
        keyphrase_ngram_range = keyphrase_ngram_range, 
        stop_words = stop_words,
        top_n = top_n,
        use_maxsum=True, 
        nr_candidates=10
        )
    print(f"MSD후 토픽 : {sorted(keywords, key=lambda x: x[1], reverse=True)} \n")

문장 : 탈원전 방향 맞 부작용 최소
원래 토픽 : [('탈원전', 0.8296), ('방향', 0.3018), ('부작용', 0.2864), ('최소', 0.2789)]
MSD후 토픽 : [] 

문장 : 탈원전 석탄 조급증 안전 값싸 전기 없
원래 토픽 : [('탈원전', 0.6194), ('전기', 0.3925), ('값싸', 0.3398), ('안전', 0.2511), ('조급증', 0.2092)]
MSD후 토픽 : [('탈원전', 0.6194), ('전기', 0.3925), ('값싸', 0.3398), ('안전', 0.2511), ('조급증', 0.2092)] 

문장 : 새 정부 탈원전 정책 우려 새기 들
원래 토픽 : [('탈원전', 0.7299), ('우려', 0.239), ('정부', 0.1664), ('새기', -0.0242), ('정책', -0.0268)]
MSD후 토픽 : [('탈원전', 0.7299), ('우려', 0.239), ('정부', 0.1664), ('새기', -0.0242), ('정책', -0.0268)] 

문장 : 탈원전 정규 직 밀어붙이 정책 놓치 것
원래 토픽 : [('탈원전', 0.7077), ('정규', 0.2489), ('놓치', 0.1947), ('밀어붙이', 0.1818), ('정책', 0.0447)]
MSD후 토픽 : [('탈원전', 0.7077), ('정규', 0.2489), ('놓치', 0.1947), ('밀어붙이', 0.1818), ('정책', 0.0447)] 



**Maximal Marginal Relevance(MMR)**

Maximal Marginal Relevance는 문서와 가장 유사한 토픽을 선정하고, 이미 선정된 토픽과 유사하지 않은 새로운 후보 중, 가장 유사한 키워드를 찾는 과정을 반복하며 토픽을 점진적으로 찾아나가는 방식입니다.

**설정해야하는 값**
- `use_mmr` : True로 설정시 해당 옵션 사용
- `diversity` : 높을수록 키워드 다양해짐(0~1)

In [102]:
for text in tokenized_text[:4]:
    print(f"문장 : {text}")
    
    keywords = kw_model.extract_keywords(
        docs = text, 
        keyphrase_ngram_range = keyphrase_ngram_range, 
        stop_words = stop_words,
        top_n = top_n
        )
    print(f"원래 토픽 : {keywords}")

    keywords = kw_model.extract_keywords(
        docs = text, 
        keyphrase_ngram_range = keyphrase_ngram_range, 
        stop_words = stop_words,
        top_n = top_n,
        use_mmr=True, 
        diversity=0.5
        )
    print(f"MMR후 토픽 : {sorted(keywords, key=lambda x: x[1], reverse=True)} \n")

문장 : 탈원전 방향 맞 부작용 최소
원래 토픽 : [('탈원전', 0.8296), ('방향', 0.3018), ('부작용', 0.2864), ('최소', 0.2789)]
MMR후 토픽 : [('탈원전', 0.8296), ('방향', 0.3018), ('부작용', 0.2864), ('최소', 0.2789)] 

문장 : 탈원전 석탄 조급증 안전 값싸 전기 없
원래 토픽 : [('탈원전', 0.6194), ('전기', 0.3925), ('값싸', 0.3398), ('안전', 0.2511), ('조급증', 0.2092)]
MMR후 토픽 : [('탈원전', 0.6194), ('전기', 0.3925), ('값싸', 0.3398), ('안전', 0.2511), ('조급증', 0.2092)] 

문장 : 새 정부 탈원전 정책 우려 새기 들
원래 토픽 : [('탈원전', 0.7299), ('우려', 0.239), ('정부', 0.1664), ('새기', -0.0242), ('정책', -0.0268)]
MMR후 토픽 : [('탈원전', 0.7299), ('우려', 0.239), ('정부', 0.1664), ('새기', -0.0242), ('정책', -0.0268)] 

문장 : 탈원전 정규 직 밀어붙이 정책 놓치 것
원래 토픽 : [('탈원전', 0.7077), ('정규', 0.2489), ('놓치', 0.1947), ('밀어붙이', 0.1818), ('정책', 0.0447)]
MMR후 토픽 : [('탈원전', 0.7077), ('정규', 0.2489), ('놓치', 0.1947), ('밀어붙이', 0.1818), ('정책', 0.0447)] 



## **수고하셨습니다**

더 많은 사용법은 [공식 GitHub](https://maartengr.github.io/BERTopic/index.html#installation)에서 찾아보세요!